In [ ]:
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langgrpah.prebuilt import create_react_agent

In [ ]:
import os
import logging

from langgraph_checkpoint_aws import AgentCoreMemorySaver
from bedrock_agentcore.memory import MemoryClient

region = os.getenv('AWS_REGION', 'us-west-2')
logging.getLogger("math-agent").setLevel(logging.DEBUG)

memory_name = "MathLanggraphAgent"
client = MemoryClient(region_name=region)
memory = client.create_or_get_memory(name=memory_name)
memory_id = memory['id']

In [ ]:
MODEL_ID = "global.anthropic.claude-haiku-4-5-20251001-v1:0"

checkpointer = AgentCoreMemorySaver(memory_id, region_name=region)

llm = init_chat_model(MODEL_ID, model_provider="bedrock_converse", region_name=region)

In [ ]:
@tool
def add(a: int, b: int):
    """Add two intergers and return the result"""
    return a + b

@tool
def multiply(a: int, b: int):
    """Multiply two integers and return the result"""
    return a * b

tools = [add, multiply]

In [ ]:
graph = create_react_agent(
    model=llm,
    tools=tools,
    prompt="You are a helpful assistant",
    checkpointer=checkpointer
)
graph

In [ ]:
config = {
    "configurable": {
        "thread_id": "session-1",
        "actor_id": "react-agent-1"
    }
}

inputs = {"messages": [{"role": "user", "content": "What is 1337 times 515321? Then add 412 and return the value to me."}]}

In [ ]:
for chunk in graph.stream(inputs, stream_mode="updates", config=config):
    print(chunk)

In [ ]:
for message in graph.get_state(config).values.get("messages"):
    print(f"{message.type}: {message.text()}")
    print("=========================================")

In [ ]:
for checkpoint in graph.get_state_history(config):
    print(
        f"(Checkpoint ID {checkpoint.config['configurable']['checkpoint_id']} of messages in state: {len(checkpoint.values.get('messages'))})"
    )

In [ ]:
inputs = {"messages": [{"role": "user", "content": "What are the first calculations I asked you to do?"}]}

for chunk in graph.stream(inputs, stream_mode="updates", config=config):
    print(chunk)

In [ ]:
config = {
    "configurable": {
        "thread-id": "session-2",
        "actor-id": "react-agent-1"
    }
}

inputs = {"messages": [{"role": "user", "content": "What values did I ask you to multiply and add?"}]}
for chunk in graph.stream(inputs, stream_mode="updates", config=config):
    print(chunk)